# OS11 - Exercises 9-11

In [1]:
from sympy import *

In [2]:
import numpy as np

In [3]:
import scipy.linalg as linalg

In [4]:
from scipy.optimize import linprog

### Previous functions useful here

In [5]:
def nodes_indexer(named_nodes):
    nodes = {}
    for i in range(len(named_nodes)):
        nodes[named_nodes[i]] = i
    return nodes

In [6]:
def ANMatrix_from_arcs(graph, num_nodes):
    num_arcs = len(graph)
    arcs = []
    MN = np.zeros((num_nodes, num_arcs)).astype(int)
    for a in range(len(graph)):
        MN[graph[a][0][0], a] = 1
        MN[graph[a][0][1], a] = -1
        arcs.append(graph[a][0])
    return MN

In [29]:
def print_path(graph, selected_nodes):
    s = ''
    sep = ' -> '
    s += sep
    for i in range(len(graph)):
        if selected_nodes[i] > 0.1:
            s += str(graph[i][0]) + sep
    return s

## Ex. 9 - SP time-restricted 

In [7]:
named_nodes = ['s', '2', '3', '4', '5', 't']   #node enumeration. Can be any non repeated inmutable.

In [8]:
nodes = nodes_indexer(named_nodes)

In [44]:
gen_graph = []

gen_graph.append((('s', '2'), (2,3)))   #distance, time
gen_graph.append((('s', '3'), (1,1)))

gen_graph.append((('2', '4'), (2,3)))
gen_graph.append((('2', 't'), (5,1)))

gen_graph.append((('3', '5'), (2,3)))

gen_graph.append((('4', 't'), (1,3)))

gen_graph.append((('5', 't'), (2,5)))

gen_graph


[(('s', '2'), (2, 3)),
 (('s', '3'), (1, 1)),
 (('2', '4'), (2, 3)),
 (('2', 't'), (5, 1)),
 (('3', '5'), (2, 3)),
 (('4', 't'), (1, 3)),
 (('5', 't'), (2, 5))]

Graph data is in named form, I need it in indexed form:

In [45]:
indexed_graph = [((nodes[a[0][0]], nodes[a[0][1]]), a[1]) for a in gen_graph]
indexed_graph

[((0, 1), (2, 3)),
 ((0, 2), (1, 1)),
 ((1, 3), (2, 3)),
 ((1, 5), (5, 1)),
 ((2, 4), (2, 3)),
 ((3, 5), (1, 3)),
 ((4, 5), (2, 5))]

Now, I can create the arcs-nodes matrix

In [46]:
A = ANMatrix_from_arcs(indexed_graph, len(nodes))
A

array([[ 1,  1,  0,  0,  0,  0,  0],
       [-1,  0,  1,  1,  0,  0,  0],
       [ 0, -1,  0,  0,  1,  0,  0],
       [ 0,  0, -1,  0,  0,  1,  0],
       [ 0,  0,  0,  0, -1,  0,  1],
       [ 0,  0,  0, -1,  0, -1, -1]])

Cost function:

In [47]:
costos = [a[1][0] for a in gen_graph]
c = np.array(costos)

c

array([2, 1, 2, 5, 2, 1, 2])

Nodes balance b:
    

In [48]:
b = np.zeros((len(nodes)))
b[nodes['s']] = 1 
b[nodes['t']] = -1 

b

array([ 1.,  0.,  0.,  0.,  0., -1.])

D.V. bounds:

In [49]:
limites = [(0, None) for a in gen_graph]
limites

[(0, None), (0, None), (0, None), (0, None), (0, None), (0, None), (0, None)]

Travel time for each arc:

In [50]:
t_j = [[a[1][1] for a in gen_graph]]
t_j = np.array(t_j)
t_j

array([[3, 1, 3, 1, 3, 3, 5]])

Maximum time available:

In [51]:
T = np.array([9])
T

array([9])

### Resolution

In [52]:
r = linprog(c=c, A_eq = A, b_eq = b, bounds = limites, A_ub= t_j, b_ub=T, method='revised simplex')

<ipython-input-52-0c828d44b360>:1: OptimizeWarning: A_eq does not appear to be of full row rank. To improve performance, check the problem formulation for redundant equality constraints.
  r = linprog(c=c, A_eq = A, b_eq = b, bounds = limites, A_ub= t_j, b_ub=T, method='revised simplex')


In [53]:
r

     con: array([0., 0., 0., 0., 0., 0.])
     fun: 5.0
 message: 'Optimization terminated successfully.'
     nit: 6
   slack: array([0.])
  status: 0
 success: True
       x: array([0., 1., 0., 0., 1., 0., 1.])

### Results

In [54]:
print_path(gen_graph, r.x)

" -> ('s', '3') -> ('3', '5') -> ('5', 't') -> "

### Second case, T=8 hs

In [55]:
T = np.array([8])
T

array([8])

In [56]:
r = linprog(c=c, A_eq = A, b_eq = b, bounds = limites, A_ub= t_j, b_ub=T, method='simplex')

<ipython-input-56-afe5f22800ef>:1: OptimizeWarning: A_eq does not appear to be of full row rank. To improve performance, check the problem formulation for redundant equality constraints.
  r = linprog(c=c, A_eq = A, b_eq = b, bounds = limites, A_ub= t_j, b_ub=T, method='simplex')


In [57]:
r

     con: array([0., 0., 0., 0., 0., 0.])
     fun: 5.4
 message: 'Optimization terminated successfully.'
     nit: 6
   slack: array([0.])
  status: 0
 success: True
       x: array([0.2, 0.8, 0. , 0.2, 0.8, 0. , 0.8])

In [58]:
r.x

array([0.2, 0.8, 0. , 0.2, 0.8, 0. , 0.8])

### Conclussions

With restriction Tmax = 9 hs problem works and solver found a solution equal to 5 hs.

When  we change restriction to 8 hs, solver fails to found a correct solution in our problem context despite being mathematically correct.

The fact that the case 8 hs is equally factible as the case 9 hs shows the resolution technique limitations.